path_to_data/raw_data/

    ├── class_0/
    │   ├── trial1
    │   │   ├── all_data.txt
    │   │   ├── true_lable.csv
    │   ├── trial2
    │   │   ├── all_data.txt
    │   │   ├── true_lable.csv
    │   └── ...
    ├── class_1/
    │   ├── trial1
    │   │   ├── all_data.txt
    │   │   ├── true_lable.csv
    │   ├── trial2
    │   │   ├── all_data.txt
    │   │   ├── true_lable.csv
    │   └── ...
    └── ...

# Franka Robot
## 1. Collect data from robots (script: frankaRobot/save_data.py) → outputs raw data
## 2. Convert raw data into labeled data

In [1]:
# class rawData2LabeledData:
import numpy as np
import pandas as pd
import os 

class rawData2LabeledData:   #make_folder_dataset:
    def __init__(self, raw_data_path:str, labeled_data_path:str, labeled_data_name:str) -> None:
        self.path = raw_data_path
        self.save_path = labeled_data_path
        self.save_name = labeled_data_name
        
        os.makedirs(self.save_path, exist_ok=True)
        self.num_lines_per_message = 130
        self.df = pd.DataFrame()
        self.tau = ['tau_J0','tau_J1', 'tau_J2', 'tau_J3', 'tau_J4', 'tau_J5', 'tau_J6']
        self.tau_d = ['tau_J_d0','tau_J_d1', 'tau_J_d2', 'tau_J_d3', 'tau_J_d4', 'tau_J_d5', 'tau_J_d6']
        self.tau_ext =['tau_ext0','tau_ext1','tau_ext2','tau_ext3','tau_ext4','tau_ext5','tau_ext6']

        self.q = ['q0','q1','q2','q3','q4','q5','q6']
        self.q_d = ['q_d0','q_d1','q_d2','q_d3','q_d4','q_d5','q_d6']

        self.dq = ['dq0','dq1','dq2','dq3','dq4','dq5','dq6']
        self.dq_d = ['dq_d0','dq_d1','dq_d2','dq_d3','dq_d4','dq_d5','dq_d6']


        self.e = ['e0','e1','e2','e3','e4','e5','e6']
        self.de = ['de0','de1','de2','de3','de4','de5','de6']
        self.etau = ['etau_J0','etau_J1', 'etau_J2', 'etau_J3', 'etau_J4', 'etau_J5', 'etau_J6']
    
    def _extract_array(self, data_dict:dict, data_frame:str, header:list,  n:int):
            dof = 7
            x, y = data_frame[n].split(':')
            y = y.replace('[','')
            y = y.replace(']','')
            y = y.replace('\n','')

            y = y.split(',')
            for i in range(dof):
                data_dict[header[i]].append(float(y[i]))

    def extract_robot_data(self):
        f = open(self.path + 'all_data.txt', 'r')
        lines = f.readlines()

        keywords = ['time'] + self.tau + self.tau_d + self.tau_ext + self.q + self.q_d + self.dq + self.dq_d 

        data_dict = dict.fromkeys(keywords)
        for i in keywords:
            data_dict[i]=[0]
        
        for i in range(int(len(lines)/self.num_lines_per_message)):
            data_frame = lines[i*self.num_lines_per_message:(i+1)*self.num_lines_per_message]
            
            x, y = data_frame[3].split(':')
            time_=int(y)

            x, y = data_frame[4].split(':')
            time_ = time_+int(y)/np.power(10,9)

            data_dict['time'].append(time_)
            
            self._extract_array(data_dict,data_frame,self.tau, 25)
            self._extract_array(data_dict,data_frame,self.tau_d, 26)
            self._extract_array(data_dict,data_frame, self.tau_ext, 37)
            
            self._extract_array(data_dict,data_frame,self.q, 28)
            
            self._extract_array(data_dict,data_frame, self.q_d, 29)
            self._extract_array(data_dict,data_frame, self.dq, 30)
            self._extract_array(data_dict,data_frame, self.dq_d, 31)
        
       
        self.df = pd.DataFrame.from_dict(data_dict)
        self.df = self.df.drop(index=0).reset_index()
        
        for i in range(len(self.e)):
            self.df[self.e[i]] = self.df[self.q_d[i]]-self.df[self.q[i]]
        for i in range(len(self.de)):
            self.df[self.de[i]] = self.df[self.dq_d[i]]-self.df[self.dq[i]]
        for i in range(len(self.etau)):
            self.df[self.etau[i]] = self.df[self.tau_d[i]]-self.df[self.tau[i]]

        #self.df.to_csv(self.save_path +'robot_data.csv',index=False)

    def get_labels(self):
        time_dev_parameter = 0.2

        true_label = pd.read_csv(self.path+'true_label.csv')

        true_label_time_digits = len(str(int(true_label['time_sec'][0])))
        
        self.df.time = self.df.time % np.power(10,true_label_time_digits)
        true_label['time'] = true_label['time_sec']+true_label['time_nsec'] - self.df['time'].iloc[0]
        
        # Compute time differences to find significant contact events
        time_dev = true_label['time'].diff()
        
        # Identify start and end indices of contact events based on the time deviation parameter
        start_times = np.append([0], true_label['time'][time_dev > time_dev_parameter].index.values)
        end_times = np.append(true_label['time'][time_dev > time_dev_parameter].index.values-1, true_label['time'].shape[0]-1)

        # Normalize 'time' in df
        self.df['time'] -= self.df['time'].iloc[0]

        self.df['label'] = 0  # Initialize the 'label' column

        # Assign labels for contact events
        for start_time, end_time in zip(start_times, end_times):
            # Create a mask for rows in df where time is within the start and end bounds
            mask = (self.df['time'] >= true_label.time[start_time]) & (self.df['time'] < true_label.time[end_time])
            self.df.loc[mask, 'label'] = 1  # Use .loc with a mask to assign label1
            
        self.true_label = true_label
        self.df.to_csv(self.save_path +self.save_name + '.csv', index=False)


In [3]:
# run on all folders within the raw_data_path
import os
raw_data_path = os.getcwd().replace('AIModels','') + 'dataset/franka_mindlab/raw_data/'
labeled_data_path = os.getcwd().replace('AIModels','') + 'dataset/franka_mindlab/labeled_data/' 
os.makedirs(labeled_data_path, exist_ok=True)
for class_name in os.listdir(raw_data_path):
    if os.path.isdir(raw_data_path+class_name):
        for trial in os.listdir(raw_data_path+class_name):
            instance = rawData2LabeledData(raw_data_path = raw_data_path+class_name+'/'+trial+'/', labeled_data_path = labeled_data_path+class_name+'/', labeled_data_name=trial)
            instance.extract_robot_data()
            instance.get_labels()

In [8]:
# plot a sample data
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

target = ['e3','tau_J3']

for i in target:
    A = instance.df[i].max()-instance.df[i].min()
    instance.df['label_scaled']=instance.df['label']*A + instance.df[i][0] -A/2
    instance.df.iplot(x='time', y= [i, 'label_scaled'], xTitle='time (sec)', yTitle=i)
    
    #plt.plot(instance.df['time'],instance.df['labeled_scaled'])

In [7]:
# run on all folders within the raw_data_path
import os
raw_data_path = os.getcwd().replace('AIModels','') + 'dataset/franka_main/raw_data/'
labeled_data_path = os.getcwd().replace('AIModels','') + 'dataset/franka_main/labeled_data/' 
os.makedirs(labeled_data_path, exist_ok=True)
for class_name in os.listdir(raw_data_path):
    if os.path.isdir(raw_data_path+class_name):
        for pose_name in os.listdir(raw_data_path+class_name):
            pose_path = f'{raw_data_path}{class_name}/{pose_name}/'
            if os.path.isdir(pose_path):
                for trial in os.listdir(pose_path):
                    instance = rawData2LabeledData(raw_data_path = f'{pose_path}/{trial}/', labeled_data_path = f'{labeled_data_path}{class_name}/', labeled_data_name=f'{trial}_{pose_name}')
                    instance.extract_robot_data()
                    instance.get_labels()

# UR Robot
## 1. Collect data from robots (script: urRobot/save_data.py) → outputs raw data
## 2. Convert raw data into labeled data

In [9]:
import numpy as np
import pandas as pd
import os 
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
import matplotlib.pyplot as plt
import seaborn as sns

cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

class rawData2LabeledData: #make_folder_dataset:
    def __init__(self, raw_data_path:str,labeled_data_path:str, labeled_data_name:str) -> None:
        self.path = raw_data_path
        self.save_path = labeled_data_path
        self.save_name = labeled_data_name
        self.dof = 6
        os.makedirs(self.save_path, exist_ok=True)

        self.df = pd.DataFrame()
        self.df_dataset = pd.DataFrame()
        self._create_dummy_data(self.dof)

    def _create_dummy_data(self,dof):
        self.target_q = [f'target_q_{i}' for i in range(dof)]
        self.actual_q = [f'actual_q_{i}' for i in range(dof)]

        self.target_qd = [f'target_qd_{i}' for i in range(dof)]
        self.actual_qd = [f'actual_qd_{i}' for i in range(dof)]

        self.target_current = [f'target_current_{i}' for i in range(dof)]
        self.actual_current = [f'actual_current_{i}' for i in range(dof)]

        self.actual_TCP_pose = [f'actual_TCP_pose_{i}' for i in range(dof)]
        self.target_TCP_pose = [f'target_TCP_pose_{i}' for i in range(dof)]

        self.actual_TCP_speed = [f'actual_TCP_speed_{i}' for i in range(dof)]
        self.target_TCP_speed = [f'target_TCP_speed_{i}' for i in range(dof)]

        self.actual_TCP_force = [f'actual_TCP_force_{i}' for i in range(dof)]
        self.joint_control_output = [f'joint_control_output_{i}' for i in range(dof)]
        self.target_moment = [f'target_moment_{i}' for i in range(dof)]

        self.joint_temperatures = [f'joint_temperatures_{i}' for i in range(dof)]
        self.actual_execution_time = ['actual_execution_time']
        self.joint_mode = [f'joint_mode_{i}' for i in range(dof)]
        self.actual_tool_accelerometer = [f'actual_tool_accelerometer_{i}' for i in range(3)]
        
        self.actual_joint_voltage = [f'actual_joint_voltage_{i}' for i in range(dof)]

        self.e = [f'e{i}' for i in range(dof)]
        self.de = [f'de{i}' for i in range(dof)]
        self.etau = [f'etau{i}' for i in range(dof)]
        self.tau_ext = [f'tau_ext{i}' for i in range(dof)]
        self.e_i = [f'e_i{i}' for i in range(dof)]

    def extract_robot_data(self):
        for file in os.listdir(self.path):
            if file.endswith(".txt"):
                df = pd.read_csv(self.path+'/'+file)

                # Extract the filename from the path
                file_name = os.path.basename(file)
                
                # Remove the file extension
                file_base = os.path.splitext(file_name)[0]

                # Convert the remaining string to a floatlace=True)
                df['time']=df['time']-df['time'][0]+self.ros_time
                #self.df.to_csv(self.save_path +'robot_data.csv',index=False)
                self.df = df

    def get_labels(self):
        time_dev_parameter = 0.2

        true_label = pd.read_csv(self.path+'true_label.csv')

        true_label_time_digits = len(str(int(true_label['time_sec'][0])))
        
        self.df.time = self.df.time % np.power(10,true_label_time_digits)
        true_label['time'] = true_label['time_sec']+true_label['time_nsec'] - self.df['time'].iloc[0]
        
        # Compute time differences to find significant contact events
        time_dev = true_label['time'].diff()
        
        # Identify start and end indices of contact events based on the time deviation parameter
        start_times = np.append([0], true_label['time'][time_dev > time_dev_parameter].index.values)
        end_times = np.append(true_label['time'][time_dev > time_dev_parameter].index.values-1, true_label['time'].shape[0]-1)

        # Normalize 'time' in df
        self.df['time'] -= self.df['time'].iloc[0]

        self.df['label'] = 0  # Initialize the 'label' column

        # Assign labels for contact events
        for start_time, end_time in zip(start_times, end_times):
            # Create a mask for rows in df where time is within the start and end bounds
            mask = (self.df['time'] >= true_label.time[start_time]) & (self.df['time'] < true_label.time[end_time])
            self.df.loc[mask, 'label'] = 1  # Use .loc with a mask to assign label1
            
        self.true_label = true_label
        self.df.to_csv(self.save_path +self.save_name + '.csv', index=False)


In [10]:
# run on all folders within the raw_data_path
import os
raw_data_path = os.getcwd().replace('AIModels','') + 'dataset/ur5/raw_data/'
labeled_data_path = os.getcwd().replace('AIModels','') + 'dataset/ur5/labeled_data/' 
os.makedirs(labeled_data_path, exist_ok=True)
for class_name in os.listdir(raw_data_path):
    if os.path.isdir(raw_data_path+class_name):
        for trial in os.listdir(raw_data_path+class_name):
            instance = rawData2LabeledData(raw_data_path = raw_data_path+class_name+'/'+trial+'/', labeled_data_path = labeled_data_path+class_name+'/', labeled_data_name=trial)
            instance.extract_robot_data()
            instance.get_labels()

In [18]:
# plot a sample data
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

target = ['e1','e_i1']

for i in target:
    A = instance.df[i].max()-instance.df[i].min()
    instance.df['label_scaled']=instance.df['label']*A + instance.df[i][0] -A/2
    instance.df.iplot(x='time', y= [i, 'label_scaled'], xTitle='time (sec)', yTitle=i)
    
    #plt.plot(instance.df['time'],instance.df['labeled_scaled'])

# Create Pickle Datasets

In [1]:
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
import torch
from torchinfo import summary 
import random

class LoadSeqDataset(Dataset):
    def __init__(self, file_path: str, label: int, selected_features:list, seq_num=28, gap=5, desire_class = None):
        """
        Initialize the dataset from a labeled data file by converting it to sequential format.
        
        Args:
            file_path (str): Path to the labeled data file.
            label (int): Label associated with the sequences from this file.
            seq_num (int): Length of each sequence.
            gap (int): Step size between sequences.
        """
        self.seq_num = seq_num
        self.gap = gap
        self.selected_features = selected_features
        # Define dtype for selected features and label
        dtypes = {col: 'float32' for col in selected_features}
        dtypes['label'] = 'int8'  # Set label as an integer type
        dtypes['time'] = 'float32'  # Set label as an integer type
        # Load data and process into sequences

        self.data = pd.read_csv(file_path, usecols=selected_features + ['label', 'time'], dtype = dtypes)
        self.data.drop(columns='index', inplace=True, errors='ignore')
        self.data.label = self.data.label * label.item()
        self.df = self.data
        #self.sequences = self._make_sequences_contact_only()
        self.sequences = self._make_sequences()
        if desire_class is not None:
            self.sequences = self._get_specific_class(desire_class)
    def balanceData(self, split_rate_1 = 1, split_rate_0=0.15):
        """Balances the dataset by downsampling sequences where label = 0 to 10%."""
        # Separate sequences based on the label
        label_0_sequences = [seq for seq in self.sequences if seq[1] == 0]
        other_label_sequences = [seq for seq in self.sequences if seq[1] != 0]

        # Downsample label 0 sequences to 10%
        num_to_keep = int(len(label_0_sequences) * split_rate_0)
        downsampled_label_0 = random.sample(label_0_sequences, num_to_keep)

        num_to_keep = int(len(other_label_sequences) * split_rate_1)
        other_label_sequences = random.sample(other_label_sequences, num_to_keep)

        # Combine and shuffle
        balanced_sequences = downsampled_label_0 + other_label_sequences
        random.shuffle(balanced_sequences)

        # Update sequences
        self.sequences = balanced_sequences

    def _make_sequences_contact_only(self):
        """Generate sequences based on the contact points detected in the data."""
        start_contact_indexs = self.df.loc[self.df.label.diff() > 0.1, :].index
        end_contact_indexs = self.df.loc[self.df.label.diff() < -0.1, :].index - 1
        contact_indexs = [idx for idx, idx2 in zip(start_contact_indexs, end_contact_indexs) if idx2 - idx >= self.seq_num]

        sequences = []
        for contact_index in contact_indexs:
            end_point = contact_index + self.seq_num
            for step in range(contact_index, end_point, self.gap):
                window = self.df[self.selected_features][step - self.seq_num + 1:step + 1]
                sequences.append((window.values, self.df.label[step]))
        return sequences
    
    def _make_sequences(self):
        """Generate sequences over time"""
        sequences = []
        for step in range(self.seq_num,self.data.shape[0], self.gap):
            window = self.df[self.selected_features][step - self.seq_num:step]
            sequences.append((window.values, self.df.label[step-1]))
        
        return sequences

    def _get_specific_class(self, desired_label):
        filtered_data = [(seq, label) for seq, label in self.sequences if label == desired_label]
        return filtered_data


    def __len__(self):
        """Return the total number of sequences in the dataset."""
        return len(self.sequences)

    def __getitem__(self, idx):
        """
        Retrieve a single sequence and label.
        
        Args:
            idx (int): Index of the sequence to retrieve.
            
        Returns:
            (tuple): (features, target) where target is the label for classification.
        """
        #TODO: multiple features should be reshaped.
        features, target = self.sequences[idx]
        features = features.T if isinstance(features, torch.Tensor) else torch.tensor(features, dtype=torch.float32).T.clone().detach()
        target = target if isinstance(target, torch.Tensor) else torch.tensor(target, dtype=torch.long).clone().detach()
        return features, target


class LoadDatasets(Dataset):
    def __init__(self, data_path:str, dict_label = None):
        """
        Load sequential dataset from a directory structure with labeled subdirectories.

            Expected directory structure:
            
            path/to/data/
                ├── class_0/
                │   ├── sample1.pkl 
                │   ├── sample2.csv
                │   └── ...
                ├── class_1/
                │   ├── sample1.csv
                │   ├── sample2.pkl
                │   └── ...
                └── ...

            Label mapping example:
            
            dict_label = {'class_0': 0, 'class_1': 1, ... }
        
        Args:
            data_path (str): Path to the data directory.
            dict_label (dict, optional): Dictionary mapping class folder names to labels.
        """
        if dict_label is None:
            dict_label = {'a': 7, 'b': 6, 'c': 5, 'd': 4, 'e': 3, 'f': 2, 'g': 1}
            #dict_label = {'link7': 7, 'link6':6, 'link5':5, 'link4':4, 'link3':3, 'link2':2, 'link1':1}

        
        self.samples = []
        self.class_to_idx = {}

        # Scan data_path for subdirectories        
        for class_name in sorted(os.listdir(data_path)):
            class_dir = os.path.join(data_path, class_name)
            if os.path.isdir(class_dir) and class_name in dict_label:
                label = dict_label[class_name]  # Look up label
                self.class_to_idx[class_name] = label
                for file_name in os.listdir(class_dir):
                    file_path = os.path.join(class_dir, file_name)
                    if os.path.isfile(file_path):
                        self.samples.append((file_path, label))
                        
    def __len__(self):
        """Return the total number of samples."""
        return len(self.samples)

    def __getitem__(self, idx):
        """
        Retrieve a single sample at the specified index.
        
        Args:
            idx (int): Index of the sample to retrieve.
            
        Returns:
            tuple: (file_path, label)
        """
        seq_path, label = self.samples[idx]
        return seq_path, label

In [ ]:
import logging
import pickle
import os
from torch.utils.data import random_split, DataLoader, Subset, ConcatDataset
# Load data 

seq_num = 80
gap = 5
split_rate = 0.3
feature = 'e'

dict_label = dict_label = {'link7': 7, 'link6':6, 'link5':5, 'link4':4, 'link3':3, 'link2':2, 'link1':1, 'no_contact': 0}
dataset_info = {
    os.getcwd().replace('AIModels', '') + '/dataset/franka_mindlab/labeled_data/': 7,
    os.getcwd().replace('AIModels', '') + '/dataset/ur5/labeled_data/': 6,
    os.getcwd().replace('AIModels', '') + '/dataset/franka_main/labeled_data/': 7
}

for data_path, dof in dataset_info.items():
    save_path = data_path.replace('labeled_data', 'pickleDatasets')
    os.makedirs(save_path, exist_ok=True)
    log_file = save_path + 'dataset_log.txt' 
    logging.basicConfig(level=logging.INFO,  # Set the logging level to INFO
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[
                        logging.StreamHandler(),  # Print log to console
                        logging.FileHandler(log_file)  # Save log to a file
                    ])

     
    selected_features = [f'{feature}{i}' for i in range(dof)]#+[ f'de{i}' for i in range(robot_dof)]
    datasets = LoadDatasets(data_path, dict_label)
    datasetsloader = DataLoader(datasets, batch_size=1, shuffle=True)
    for seq_num in range(25,200, 5):
        for gap in range(1, 21, 1):
            # Initialize counters
            total_samples_counter = 0
            train_samples_counter = 0
            master_dataset = []

            # Iterate through test_datasetloader
            for trial_dataset_path, label in datasetsloader:
                #if 't2.csv' in trial_dataset_path[0] or 't1.csv' in trial_dataset_path[0]:
                    # Load the dataset for the trial
                    data = LoadSeqDataset(file_path=trial_dataset_path[0], label=label[0], selected_features=selected_features, seq_num=seq_num, gap=gap)

                    total_samples = len(data)
                    total_samples_counter += total_samples

                    data.balanceData(split_rate_1=split_rate, split_rate_0=split_rate*0.1)
                    # Calculate number of samples for the subset (5% of the data)
                    num_samples = len(data)
                    train_samples_counter += num_samples
                    master_dataset.append(data)

            # Combine all subsets into a single dataset
            master_dataset = ConcatDataset(master_dataset)
            data_path_split = data_path.split('/')
            save_name = f'{data_path_split[len(data_path_split)-3]}_feature_{feature}_gap_{gap}_splitRate_{split_rate}_seqNum_{seq_num}.pickle'

            logging.info(f'------------------{save_name}------------------')
            logging.info(f"Total samples across all trials: {total_samples_counter}")
            logging.info(f"Total samples used for training ({train_samples_counter/total_samples_counter*100}% of data): {train_samples_counter}")
            train_dataloader = DataLoader(master_dataset, batch_size=10000, shuffle=True)
            unique_labels, counts = 0 , 0
            for X_batch, y_batch in train_dataloader:
                unique_labels, b = y_batch.unique(return_counts=True)
                counts += b
            logging.info(f"Unique labels: {unique_labels}")
            logging.info(f"Counts: {counts}")

            
            # Save the dataset to a file
            with open(save_path+ save_name, 'wb') as f:
                pickle.dump(master_dataset, f)
